## 3.1 Python's Functions Are First-Class

Function objects and their names are two separate concerns. Here’s more proof: You can delete the function’s original name (yell). Since another name (bark) still points to the underlying function, you can still call the function through it:

In [1]:
def yell(text):
    return text.upper() + '!'

In [2]:
bark = yell

In [3]:
bark('woof')

'WOOF!'

In [4]:
del yell

yell('hello?')

NameError: name 'yell' is not defined

In [5]:
bark('hey')

'HEY!'

Here’s how you might format a sequence of greetings all at once by mapping the bark function to them:

In [7]:
list(map(bark, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

return a function will not be executed immediately

In [8]:
def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...' 
    def yell():
        return text.upper() + '!' 
    if volume > 0.5:
        return yell 
    else:
        return whisper

In [9]:
get_speak_func('Hello, World', 0.7)

<function __main__.get_speak_func.<locals>.yell()>

In [10]:
get_speak_func('Hello, World', 0.7)()

'HELLO, WORLD!'

Objects Can Behave Like Functions:
    
This is all powered by the __call__ dunder method. Here’s an example of class defining a callable object:

In [11]:
class Adder:
    def __init__(self, n):
        self.n = n
    def __call__(self, x):
        return self.n + x

In [12]:
plus_3 = Adder(3)

In [13]:
plus_3(4)

7

In [14]:
callable(plus_3)

True

## 3.2 Lambdas Are Single-Expression Function

In [15]:
(lambda x, y: x + y)(5, 3)

8

pass to sort function

In [16]:
tuples = [(1, 'd'), (2, 'b'), (4, 'a'), (3, 'c')]
sorted(tuples, key=lambda x: x[1])

[(4, 'a'), (2, 'b'), (3, 'c'), (1, 'd')]

In [17]:
sorted(range(-5, 6), key=lambda x: x * x)

[0, -1, 1, -2, 2, -3, 3, -4, 4, -5, 5]

I have similar feelings about complicated map() or filter() constructs using lambdas. Usually it’s much cleaner to go with a list comprehension or generator expression:

In [18]:
# Harmful:
list(filter(lambda x: x % 2 == 0, range(16)))

[0, 2, 4, 6, 8, 10, 12, 14]

In [20]:
# Better:
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

## 3.3 The Power of Decorators

In [22]:
# wrapper 外面再套一层 uppercase，是因为不想让 wrapper 立即执行
def uppercase(func):
    def wrapper():
        original_result = func() 
        modified_result = original_result.upper() 
        return modified_result 
    return wrapper

In [23]:
@uppercase 
def greet():
    return 'Hello!'

In [24]:
greet()

'HELLO!'

In [29]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() ' 
              f'with {args}, {kwargs}')

        original_result = func(*args, **kwargs)

        print(f'TRACE: {func.__name__}() ' 
              f'returned {original_result!r}')

        return original_result 
    return wrapper

In [30]:
@trace 
def say(name, line):
    return f'{name}: {line}'

say('Jane', 'Hello, World')

TRACE: calling say() with ('Jane', 'Hello, World'), {}
TRACE: say() returned 'Jane: Hello, World'


'Jane: Hello, World'

How to write 'debuggable' decorators.

The awkward situation for debugging after decorate

In [31]:
def greet():
    """Return a friendly greeting.""" 
    return 'Hello!'

decorated_greet = uppercase(greet)

In [32]:
greet.__name__

'greet'

In [33]:
greet.__doc__

'Return a friendly greeting.'

In [34]:
decorated_greet.__name__

'wrapper'

In [35]:
decorated_greet.__doc__

solve this

In [36]:
import functools

def uppercase(func):
    @functools.wraps(func) 
    def wrapper():
        return func().upper() 
    return wrapper

In [37]:
@uppercase
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

greet.__name__

'greet'

In [38]:
greet.__doc__

'Return a friendly greeting.'

## 3.4 Fun With \*args and \**kwargs

If we call the function with additional arguments, args will collect extra positional arguments as a tuple because the parameter name has a * prefix.

Likewise, kwargs will collect extra keyword arguments as a dictionary because the parameter name has a ** prefix.

In [39]:
def foo(required, *args, **kwargs):
    print(required) 
    if args:
        print(args) 
    if kwargs:
        print(kwargs)

In [40]:
foo()

TypeError: foo() missing 1 required positional argument: 'required'

In [41]:
foo('hello')

hello


In [42]:
foo('hello', 1, 2, 3)

hello
(1, 2, 3)


In [43]:
foo('hello', 1, 2, 3, key1='value', key2=999)

hello
(1, 2, 3)
{'key1': 'value', 'key2': 999}


This also gives you an opportunity to modify the arguments before you pass them along. Here’s an example:

In [44]:
def foo(x, *args, **kwargs):
    kwargs['name'] = 'Alice'
    new_args = args + ('extra', ) 
    bar(x, *new_args, **kwargs)

In [46]:
def trace(f):
    @functools.wraps(f) 
    def decorated_function(*args, **kwargs):
        print(f, args, kwargs)
        result = f(*args, **kwargs)
        print(result) 
    return decorated_function


@trace 
def greet(greeting, name):
    return '{}, {}!'.format(greeting, name)

In [47]:
greet('Hello', 'Bob')

<function greet at 0x7f976d759310> ('Hello', 'Bob') {}
Hello, Bob!


## 3.5 Function Argument Unpacking

In [49]:
def print_vector(x, y, z):
    print('<%s, %s, %s>' % (x, y, z))

In [50]:
tuple_vec = (1, 0, 1)
list_vec = [1, 0, 1]
print_vector(tuple_vec[0], 
             tuple_vec[1], 
             tuple_vec[2])

<1, 0, 1>


Thankfully, there’s a better way to handle this situation in Python with Function Argument Unpacking using the * operator:

In [51]:
print_vector(*tuple_vec)

<1, 0, 1>


In [52]:
print_vector(*list_vec)

<1, 0, 1>


In [53]:
genexpr = (x * x for x in range(3))
print_vector(*genexpr)

<0, 1, 4>


We could pass this dict to print_vector in much the same way using the ** operator for unpacking:

In [54]:
dict_vec = {'y': 0, 'z': 1, 'x': 1}
print_vector(**dict_vec)

<1, 0, 1>


If you were to use the single asterisk (*) operator to unpack the dictionary, keys would be passed to the function in random order instead:

In [55]:
print_vector(*dict_vec)

<y, z, x>


## 3.6 Nothing to Return Here